<a href="https://colab.research.google.com/github/Sujangyawali/Fraud_Detection/blob/master/eight_class_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install keras==2.1.0 
!pip install tensorflow==1.15.0

In [ ]:
!python setup.py install

In [ ]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob
from imgaug import augmenters as iaa
import scipy.misc
import matplotlib

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn.model import MaskRCNN


# define the test configuration
class TestConfig(Config):
     NAME = "test"
     GPU_COUNT = 1
     IMAGES_PER_GPU = 1
     NUM_CLASSES = 9
    
# define 81 classes that the coco model knowns about
class_names = ['BG', 'Fire_truck','Ambulance','hiace','bus','truck','car','jeep','bike']


model = MaskRCNN(mode='inference', model_dir='./', config=TestConfig())



In [ ]:
model.load_weights('D:\\projects\\automatic _traffic_system\\models_for_eight_class\\mask_rcnn_eight_class_cfgh_0016.h5', by_name=True)

In [ ]:
import cv2
import numpy as np


def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image
def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    colors = random_colors(n_instances)

    if not n_instances:
         print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image

In [ ]:
#for two lane 




def list_contains(List1, List2): 
  
    set1 = set(List1) 
    set2 = set(List2) 
    if set1.intersection(set2): 
        return True 
    else: 
        return False

import cv2 as cv
cap = cv.VideoCapture('D:\\projects\\automatic _traffic_system\\picture\\ambulance1.mp4')
cap1 = cv.VideoCapture('D:\\projects\\automatic _traffic_system\\picture\\woe2.mp4')

size = (
    int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
)


codec = cv2.VideoWriter_fourcc(*'DIVX')

#output1 = cv2.VideoWriter('D:\projects\automatic _traffic_system\tomorrow\\1lane1.avi', codec, 1.0, size)
#output2 = cv2.VideoWriter('D:\projects\automatic _traffic_system\tomorrow\\1lane2.avi', codec, 1.0, size)



if (cap.isOpened()== False):
    print("Error camera 1 isn't connecting")
if (cap1.isOpened()== False):
    print("Error camera 2 isn't connecting")
currentframe1=0
currentframe2=0
while (cap.isOpened() or cap1.isOpened()):
    ret, img = cap.read()
    ret1, img1 = cap1.read()
    
    if ret == True:
        currentframe1 +=1
        #print(currentframe1)
        if ((currentframe1%30)==0):
                results1 = model.detect([img], verbose=0)
                r1 = results1[0]
                img = display_instances(
                img, r1['rois'], r1['masks'], r1['class_ids'], class_names, r1['scores'])
                #output1.write(img)
                #cv2.imshow('lane1',img )
                print('lane1',r1['class_ids'])
                class_ids1 = r1['class_ids']
                emergency=[1,2]
                if list_contains(class_ids1, emergency):
                    print('Emergency vehicle detected')
                    print('Green light is on for lane 1')
                    continue
                
                else:
                    density_of_lane1=len(class_ids1)
                    print('Number of vehicle in lane 1:',len(class_ids1))
               

            
                results2 = model.detect([img1], verbose=0)
                r2 = results2[0]
                img1 = display_instances(
                img1, r2['rois'], r2['masks'], r2['class_ids'], class_names, r2['scores'])
                #output2.write(img1)
                #cv2.imshow('lane2', img1)
                print('lane2',r2['class_ids'])
                class_ids2 = r2['class_ids']
                emergency=[1,2]
                if list_contains(class_ids2, emergency):
                    print('Emergency vehicle detected')
                    print('Green light is on for lane 2')
                    continue
                    
                else:
                    density_of_lane2=len(class_ids2)
                    print('Number of vehicle in lane 2:',len(class_ids2))
                        
                if (density_of_lane1>density_of_lane2):
                    print('green light on for lane 1')
                else:
                    print('green light on for lane 2')
                        
                




    if  cv.waitKey(1) & 0xFF == ord('q'):
        break
   


cap.release()
cap1.release()
cv.destroyAllWindows()